In [ ]:
## Start Spark Application
import pyspark

In [ ]:
## Create Dataframes to process CSV
taxidata = spark.read.format('csv').options(header='true', inferSchema='true').load('s3a://dakindre-nyc-taxidata/taxi-data/*.csv')
taxiinfo = spark.read.format('csv').options(header='true', inferSchema='true').load('s3a://dakindre-nyc-taxidata/taxi-info/taxi _zone_lookup.csv')

In [ ]:
## Perform Transformations on the data
taxidata2 = taxidata.select('tpep_pickup_datetime', 'tpep_dropoff_datetime', 'trip_distance', 'PULocationID', 'DOLocationID')

mergedTaxi = taxidata2.join(taxiinfo, taxidata.PULocationID == taxiinfo.LocationID)
mergedTaxiPU = mergedTaxi.withColumnRenamed("locationid", "pu2locationid").withColumnRenamed("borough", "puborough").withColumnRenamed("zone", "puzone").withColumnRenamed("service_zone", "puservice_zone")

mergedTaxi2 = mergedTaxiPU.join(taxiinfo, taxidata.DOLocationID == taxiinfo.LocationID)
mergedTaxiDO = mergedTaxi2.withColumnRenamed("locationid", "do2locationid").withColumnRenamed("borough", "doborough").withColumnRenamed("zone", "dozone").withColumnRenamed("service_zone", "doservice_zone")

mergedFinal = mergedTaxiDO.drop('do2locationid').drop('pu2locationid')

In [ ]:
mergedFinal.show(10)

In [ ]:
mergedFinal.write.parquet("s3a://dakindre-nyc-taxidata/taxi-data-etl/",mode="overwrite")

In [ ]:
spark.stop()